# validate t_part

compare t_part with [Kangxi 214 radicals](https://www.xiaoma.info/bushou.php?ext=1)

In [1]:
from utils_copy import *

from IPython.display import Image

In [37]:
sql_stmt = f"""
select --zi, 
case when --coalesce(alias,traditional) is not null  and 
	coalesce(alias,traditional) != '' then 
	zi || '(' || coalesce(alias,traditional) || ')' 
	ELSE zi
end as Radicaal, 
strokes, pinyin, meaning as definition 
from t_part 
order by cast(strokes as int), 
zi;
"""
with DBConn() as _conn:
    df1 = pd.read_sql(sql_stmt, _conn)

df1 

,Radicaal,strokes,pinyin,definition
0,丬,None,None,None
1,一,1,yī,one
2,丨,1,shù,vertical line Through Alone Straight
3,丶,1,zhǔ,dot / point
4,丿(乀乁),1,piě,slash
...,...,...,...,...
409,畺,13,jiāng,None
410,鼓,13,gǔ,drum
411,鼠,13,shǔ,rat
412,鼻,14,bí,nose


In [38]:
df1.to_csv('t_part_radicals.csv', index=False)

## read table from HTML

below code is provided by Claude 3.5 sonnet on 2024-07-07

```
Using python, How can one read and parse the HTML table from this URL : "https://www.xiaoma.info/bushou.php?ext=1" and save it to pandas dataframe
```

In [29]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# # Send a GET request to the URL
# url = "https://www.xiaoma.info/bushou.php?ext=1"
# response = requests.get(url)

# Parse the HTML content
# soup = BeautifulSoup(response.content, 'html.parser')

## because the page resist scraping, save the page manually to a local HTML file

# Read the local HTML file
with open('Xiaoma Cidian - Kangxi Radicals.html', 'r', encoding='utf-8') as file:
    html_content = file.read()


# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table in the HTML
table = soup.find('table', {'class': 'radicals'})

trows = table.find_all('tr')

headers = [td.text.strip() for td in trows[0].find_all('td')]

# Extract table rows
rows = []
for tr in trows[1:]:  # Skip the header row
    row = [td.text.strip() for td in tr.find_all('td')]
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows, columns=headers)

df = df[df.columns[1:]]  # skip index column

## df.head()

# Save the DataFrame to a CSV file (optional)
df.to_csv('kanxi_radicals.csv', index=False)

In [36]:
df

,Radical,Strokes,Pinyin,Definition
0,一,1,yī,one
1,丨,1,gǔn,line
2,丶,1,zhǔ,dot
3,丿 (乀乁),1,piě,slash
4,乙 (乚乛),1,yǐ,second
...,...,...,...,...
209,齊 (齐),14,qí,neat
210,齒 (齿),15,chǐ,teeth
211,龍 (龙),16,lóng,dragon
212,龜 (龟),16,guī,turtle


## Dedup parts

In [18]:
s = """ 
|[ 1 ]| 乚一乙丶丨乀乁乛亅丿 |[ 2 ]| 亠又丷儿勹匕冫讠几刁乂龴刀刂匸丁九七八卜二㔾匚厂⺀⺁卩冂

凵了力冖乃人亻入十厶 |[ 3 ]| 土丸尢亡囗兀夕下乡上小忄已弋廴丈之夂子才纟艹阝门⻏大马山川

辶寸习勺刃也叉乞千万亏飞久于及个工弓廾与义亍夊己彐⺌⺍尸巾屮巛口干幺广彑么彳宀女

丬犭三彡饣士扌氵 |[ 4 ]| 天气瓦王文五午勿见心爻爿片牙牜⺧以尤友中元月长爫支止爪巴礻禸罓

贝耂木不车韦风巨尺云歹冈井丰夫仓勾公区匀六斗今内分丑少方凶升太为匹乌亢介父屯戈专

开办毋⺩火灬斤毛户戶牛攴攵欠无犬旡日曰朩氏手殳水 |[ 5 ]| 比田罒未氺兄穴央衤玄用玉乍𤴔石

白半包北本必丙癶布钅斥鸟出龙由旦电头乐台术业市击冬四古东失申正宁平戋末只丘占句可

甘去它甲尼刍乎瓜兰禾目卡立令另矛母疒皮且皿生矢示疋 |[ 6 ]| 西囟血羊页衣羽聿至舟⺮竹自网

⺶⺷艮虫行覀臣齐屰共曲庄向交吉而耳缶各合并尧虍臼耒吕米糸肉色舌式寺 |[ 7 ]| 辛言酉肙豸走

足㐬坙舛釆角谷邑里辰赤卤麦龟员豆良夆孚甫佥呙奂㕻身豕 |[ 8 ]| 易鱼雨隹東幷非阜隶齿黾奉其

尚责龺金青 |[ 9 ]| 畏咸音面韭食香鬼畐革柬骨首
"""

In [23]:
from collections import Counter


strokes = []
for i in s.split("|["):
    i = i.strip()
    if not i: continue
        
    k = i.split("]|")
    strokes.append([k[0].strip(), k[1].strip()])

In [24]:
strokes

[['1', '乚一乙丶丨乀乁乛亅丿'],
 ['2', '亠又丷儿勹匕冫讠几刁乂龴刀刂匸丁九七八卜二㔾匚厂⺀⺁卩冂\n\n凵了力冖乃人亻入十厶'],
 ['3',
  '土丸尢亡囗兀夕下乡上小忄已弋廴丈之夂子才纟艹阝门⻏大马山川\n\n辶寸习勺刃也叉乞千万亏飞久于及个工弓廾与义亍夊己彐⺌⺍尸巾屮巛口干幺广彑么彳宀女\n\n丬犭三彡饣士扌氵'],
 ['4',
  '天气瓦王文五午勿见心爻爿片牙牜⺧以尤友中元月长爫支止爪巴礻禸罓\n\n贝耂木不车韦风巨尺云歹冈井丰夫仓勾公区匀六斗今内分丑少方凶升太为匹乌亢介父屯戈专\n\n开办毋⺩火灬斤毛户戶牛攴攵欠无犬旡日曰朩氏手殳水'],
 ['5',
  '比田罒未氺兄穴央衤玄用玉乍𤴔石\n\n白半包北本必丙癶布钅斥鸟出龙由旦电头乐台术业市击冬四古东失申正宁平戋末只丘占句可\n\n甘去它甲尼刍乎瓜兰禾目卡立令另矛母疒皮且皿生矢示疋'],
 ['6', '西囟血羊页衣羽聿至舟⺮竹自网\n\n⺶⺷艮虫行覀臣齐屰共曲庄向交吉而耳缶各合并尧虍臼耒吕米糸肉色舌式寺'],
 ['7', '辛言酉肙豸走\n\n足㐬坙舛釆角谷邑里辰赤卤麦龟员豆良夆孚甫佥呙奂㕻身豕'],
 ['8', '易鱼雨隹東幷非阜隶齿黾奉其\n\n尚责龺金青'],
 ['9', '畏咸音面韭食香鬼畐革柬骨首']]

In [31]:
zi_parts = []

for x in strokes: 
    for i in x[1]:
        i = i.strip()
        if not i: continue
        zi_parts.append([i,x[0]])

In [33]:
len(zi_parts), zi_parts  # 410

(410,
 [['乚', '1'],
  ['一', '1'],
  ['乙', '1'],
  ['丶', '1'],
  ['丨', '1'],
  ['乀', '1'],
  ['乁', '1'],
  ['乛', '1'],
  ['亅', '1'],
  ['丿', '1'],
  ['亠', '2'],
  ['又', '2'],
  ['丷', '2'],
  ['儿', '2'],
  ['勹', '2'],
  ['匕', '2'],
  ['冫', '2'],
  ['讠', '2'],
  ['几', '2'],
  ['刁', '2'],
  ['乂', '2'],
  ['龴', '2'],
  ['刀', '2'],
  ['刂', '2'],
  ['匸', '2'],
  ['丁', '2'],
  ['九', '2'],
  ['七', '2'],
  ['八', '2'],
  ['卜', '2'],
  ['二', '2'],
  ['㔾', '2'],
  ['匚', '2'],
  ['厂', '2'],
  ['⺀', '2'],
  ['⺁', '2'],
  ['卩', '2'],
  ['冂', '2'],
  ['凵', '2'],
  ['了', '2'],
  ['力', '2'],
  ['冖', '2'],
  ['乃', '2'],
  ['人', '2'],
  ['亻', '2'],
  ['入', '2'],
  ['十', '2'],
  ['厶', '2'],
  ['土', '3'],
  ['丸', '3'],
  ['尢', '3'],
  ['亡', '3'],
  ['囗', '3'],
  ['兀', '3'],
  ['夕', '3'],
  ['下', '3'],
  ['乡', '3'],
  ['上', '3'],
  ['小', '3'],
  ['忄', '3'],
  ['已', '3'],
  ['弋', '3'],
  ['廴', '3'],
  ['丈', '3'],
  ['之', '3'],
  ['夂', '3'],
  ['子', '3'],
  ['才', '3'],
  ['纟', '3'],
  ['艹', '3'],
  ['阝', '3'],


In [35]:
!dir elemental_zi_v2.csv

 Volume in drive C is Windows
 Volume Serial Number is 6EA7-3DCA

 Directory of C:\Users\p2p2l\projects\wgong\zistory\zinets\dev\notebook

01/25/2025  10:39 PM             2,884 elemental_zi_v2.csv
               1 File(s)          2,884 bytes
               0 Dir(s)  156,147,916,800 bytes free


In [34]:
file_csv = "elemental_zi_v2.csv"
import pandas as pd 
df = pd.DataFrame(zi_parts, columns=["zi", "nstrokes"])
df.to_csv(file_csv, index=False)

In [25]:
!cd 

C:\Users\p2p2l\projects\wgong\zistory\zinets\dev\notebook


In [ ]:
    for j in i: 
        j = j.strip()
        if not j: continue
        zis.append(j.strip())

char_counts = Counter(zis)

# char_counts

for k,v in char_counts.items():
    if v > 1:
        print(k)

In [19]:
def find_duplicate_chars(text):
    """Finds and prints duplicate characters in a given text string.

    Args:
        text: The input text string, potentially containing newlines.
    """
    if not isinstance(text, str):
        raise TypeError("Input must be a string.")

    char_counts = Counter(text)  # Directly count characters in the string

    duplicates = [char for char, count in char_counts.items() if count > 1] #list comprehension for more concise code
    if duplicates:
        print("Duplicate characters:")
        for char in duplicates:
            print(char)
        return duplicates
    else:
        print("No duplicate characters found.")
        return []

In [20]:
find_duplicate_chars(s)

Duplicate characters:
 


|
[
]


[' ', '\n', '|', '[', ']']